In [7]:
import os
import pandas as pd

In [10]:
path = r"C:\Users\Krul\Documents\GitHub\cell-painting-dashboard\data\processed\resized_merged_images"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
cells = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        cells.append(file.name)

In [24]:
img_df = pd.DataFrame(cells, columns = ['FileNames'])
img_df['Row'] = img_df['FileNames'].apply(lambda x: chr(64 + int(x[1:3])))
img_df['Column'] = img_df['FileNames'].apply(lambda x: x[4:6])
img_df['F'] = img_df['FileNames'].apply(lambda x: x[7:9]) #IDK what is this
# img_df['P'] = img_df['FileNames'].apply(lambda x: x[10:12]) #Idk what is this
# img_df['Channel'] = img_df['FileNames'].apply(lambda x: x[15:16])
# img_df['Rest'] = img_df['FileNames'].apply(lambda x: x[16:-5])

In [26]:
img_df['Well'] = img_df['Row'] + img_df['Column']

In [29]:
img_df = img_df.drop(['Row', 'Column', 'F'], 1)

C:\Users\Krul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [30]:
img_df

,FileNames,Well
0,r01c01f01p01-sk1fk1fl1.tiff,A01
1,r01c01f02p01-sk1fk1fl1.tiff,A01
2,r01c01f03p01-sk1fk1fl1.tiff,A01
3,r01c01f04p01-sk1fk1fl1.tiff,A01
4,r01c01f05p01-sk1fk1fl1.tiff,A01
...,...,...
859,r16c24f05p01-sk1fk1fl1.tiff,P24
860,r16c24f06p01-sk1fk1fl1.tiff,P24
861,r16c24f07p01-sk1fk1fl1.tiff,P24
862,r16c24f08p01-sk1fk1fl1.tiff,P24


In [9]:
well_df = pd.read_csv('E:\data\Cell-Painting-HepG2-Plate-Layouts.csv', usecols=[0,1,2])
well_df 

,Well,Compound,Concentration [uM]
0,A01,Berberine Chloride,50.000000
1,A02,Berberine Chloride,25.000000
2,A03,Berberine Chloride,12.500000
3,A04,Berberine Chloride,6.250000
4,A05,Berberine Chloride,3.166667
...,...,...,...
379,P20,Tetrandrine,0.166667
380,P21,Tetrandrine,0.083333
381,P22,Tetrandrine,0.041667
382,P23,DMSO,NaN


In [33]:
df = img_df.merge(well_df, how='inner', on='Well')
df = df.drop('Concentration [uM]', 1)
df

C:\Users\Krul\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,FileNames,Well,Compound
0,r01c01f01p01-sk1fk1fl1.tiff,A01,Berberine Chloride
1,r01c01f02p01-sk1fk1fl1.tiff,A01,Berberine Chloride
2,r01c01f03p01-sk1fk1fl1.tiff,A01,Berberine Chloride
3,r01c01f04p01-sk1fk1fl1.tiff,A01,Berberine Chloride
4,r01c01f05p01-sk1fk1fl1.tiff,A01,Berberine Chloride
...,...,...,...
859,r16c24f05p01-sk1fk1fl1.tiff,P24,DMSO
860,r16c24f06p01-sk1fk1fl1.tiff,P24,DMSO
861,r16c24f07p01-sk1fk1fl1.tiff,P24,DMSO
862,r16c24f08p01-sk1fk1fl1.tiff,P24,DMSO


In [1]:
from tqdm import tqdm
import gc
import math
import random
import os
import pandas as pd
import numpy as np

# Visuals and CV2
import matplotlib.pyplot as plt
# import cudf, cuml, cupy
import cv2

# albumentations for augs
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

#torch
import torch
import timm
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import warnings 
warnings.filterwarnings('ignore')

C:\Users\Krul\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Krul\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Krul\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


ModuleNotFoundError: No module named 'torchvision.transforms'

In [2]:
import torch
print(torch.__version__)

1.7.1+cpu
